### 2.1  
  In $\epsilon$-greedy action selection, for the case of two actions and $\epsilon$ = 0.5, what is the probability that the greedy action is selected?
  
  Since the value for otimal action is $$ (1-\epsilon) + \frac {\epsilon}{n} $$ where n is number of possible action, it will result in $$ (1-0.5) + \frac {0.5}{2} = 0.75 $$

### 2.2
  *Bandit example* Consider a $k$ -armed bandit problem with k = 4 actions,
  denoted $1, 2, 3,$ and $4$. Consider applying to this problem a bandit algorithm using
  $\epsilon$-greedy action selection, sample-average action-value estimates, and initial estimates
  of $Q_1 (a) = 0$, for all a. Suppose the initial sequence of actions and rewards is $A_1 = 1$,
  $R_1 = 1$, $A_2 = 2$, $R_2 = 1$, $A_3 = 2$, $R_3 = 2$, $A_4 = 2$, $R_4 = 2$, $A_5 = 3$, $R_5 = 0$. On some
  of these time steps the " case may have occurred, causing an action to be selected at
  random. On which time steps did this definitely occur? On which time steps could this
  possibly have occurred?  

In [ ]:
import matplotlib.pyplot as plt

xs = [i for i in range(1,6)]
ys = [-1,1,-2,2,0]
fig, ax = plt.subplots()
ax.scatter(xs, ys, c='r')
ax.set_xlabel("Timestep")
ax.set_ylabel("Reward")
plt.savefig('2_2.jpg')

![Solution](2_2.jpg)

It definitely occured on timesteps 3,4 and 5

### 2.3 
  In the comparison shown in Figure 2.2, which method will perform best in
  the long run in terms of cumulative reward and probability of selecting the best action?
  How much better will it be? Express your answer quantitatively.

  In terms of average reward method with $\epsilon = 0.01$ seems to not yet level out in performance,
  as well as it didn't seem to level out in optimal action $%$, so it will probably overcome
  the other eps-greedy method because it will select the most rewarding action which they probably
  already figured out but are still choosing to explore, so I belive that because of $\epsilon$
  0.1 method will approach average reward of $1.54 \cdot 0.9$ and the other $1.54 \cdot 0.99$ and 
  the optimal action $%$ will be the same story

### 2.4
  If the step-size parameters, $a_n$ are not constant, then the estimate $Q_n$ is
  a weighted average of previously received rewards with a weighting different from that
  given by (2.6). What is the weighting on each prior reward for the general case, analogous
  to (2.6), in terms of the sequence of step-size parameters?

  $ R_{t}=G_{t-1}-\alpha G_t$

### Exercise 2.5 (programming)
Design and conduct an experiment to demonstrate the
difficulties that sample-average methods have for nonstationary problems. Use a modified
version of the 10-armed testbed in which all the $q_{*}(a)$ start out equal and then take
independent random walks (say by adding a normally distributed increment with mean 0
and standard deviation 0.01 to all the $q_{*}(a)$ on each step). Prepare plots like Figure 2.2
for an action-value method using sample averages, incrementally computed, and another
action-value method using a constant step-size parameter, $\alpha = 0.1$. Use $\epsilon = 0.1$ and
longer runs, say of 10,000 steps.

In [ ]:
#2.5
from numpy import zeros, argmax, random, empty
from numpy.random import normal, randint, random as rand
steps, episodes, alpha, eps, n_levers = 10000, 2000, 0.1, 0.1, 10
Qs = [np.zeros(n_levers) for _ in range(episodes)]
ns = [np.zeros(n_levers) for _ in range(episodes)]
qs = [np.array([normal(randint(-2,3),1) for _ in range(n_levers)]) for _ in range(episodes)]
avgrwd = [zeros(steps, dtype=float) for _ in range(episodes)]
bstrwd = [zeros(steps, dtype=float) for _ in range(episodes)]
peropt = [zeros(steps, dtype=float) for _ in range(episodes)]

for e in range(episodes):
    for i in range(steps):
        act = argmax(Qs[e]) if eps < rand() else randint(n_levers)
        ns[e][act] += 1
        Qs[e][act] += (qs[e][act] - Qs[e][act]) / ns[e][act]
        avgrwd[e][i] = (avgrwd[e][i-1] * i + qs[e][act]) / (i+1)
    #    print(f"act={act}, Qs[act]={Qs[act]:+.5f}, argmax(Qs)={argmax(Qs)}\
    #, argmax(qs)={argmax(qs)}, max(qs)={max(qs):+.5f}, avgrwd[i]={avgrwd[i]:+.5f}, best action chosen: {act == argmax(qs)}")
        bstrwd[e][i] = max(qs[e])
        peropt[e][i] = (peropt[e][i-1] * i + (act == argmax(qs[e]))) / (i+1)
        for j in range(len(qs[e])): qs[e][j] += normal(0, 0.01)
rewards, best_rewards, percent = np.mean(avgrwd, axis=0), np.mean(bstrwd, axis=0), np.mean(peropt, axis=0)

![Rozwiazanie](2_5.png)

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(nrows=2)
fig.set_dpi(120)
ax1.plot(range(steps), rewards, c='r', label='Average Reward')
ax1.axhline(best_rewards.mean(), c='b', label='Mean Best Reward')
ax1.legend()
ax2.plot(range(steps), percent, c='r', label='% optimal action')
ax2.legend()
plt.show()

### 2.6  
*Mysterious Spikes* The results shown in Figure 2.3 should be quite reliable
because they are averages over 2000 individual, randomly chosen 10-armed bandit tasks.
Why, then, are there oscillations and spikes in the early part of the curve for the optimistic
method? In other words, what might make this method perform particularly better or
worse, on average, on particular early steps?

It's because all of our possible actions have their values way above what's possible, so we will choose
the best possible action at the start, but won't acknowledge that it is indeed the best choice, since all the other ones
have their value averages way to big.

### 2.7
*Unbiased Constant-Step-Size Trick* In most of this chapter we have used
sample averages to estimate action values because sample averages do not produce the
initial bias that constant step sizes do (see the analysis leading to (2.6)). However, sample
averages are not a completely satisfactory solution because they may perform poorly
on nonstationary problems. Is it possible to avoid the bias of constant step sizes while
retaining their advantages on nonstationary problems? One way is to use a step size of
$$\beta_{n} = \alpha~/~\overline{o}_{n} ~~~~(2.8)$$
to process the nth reward for a particular action, where $\alpha > 0$ is a conventional constant
step size, and $ō$ n is a trace of one that starts at $0$:
.
.
$$\bar{o}_n = \bar{o}_{n-1}~+~\alpha(1~-~\bar{o}_{n-1} ),~~for~ n \ge 0,~~ with~~ \bar o_{0} = 0.~~~~(2.9)$$

Carry out an analysis like that in (2.6) to show that $Q_n$ n is an exponential recency-weighted
average *without initial bias*

$ Q_{n+1} = Q_n + \alpha/\bar{o}_n[R_n - Q_n] $

$ Q_{n+1} = \alpha/\bar{o}_n R_n + (1-\alpha/\bar{o}_n)Q_n $

$ Q_{n+1} = \alpha/\bar{o}_n R_n + (1-\alpha/\bar{o}_n)[\alpha/\bar{o}_n
R_{n-1} + (1-\alpha/\bar{o}_{n-1})Q_{n-1}]$

$ Q_{n+1} = \alpha/\bar{o}_n R_n + (1-\alpha/\bar{o}_n)\alpha/\bar{o}_n $
$ R_{n-1} + (1-\alpha/\bar{o}_{n-1})^{2}Q_{n-1} $

$ Q_{n+1} = \alpha/\bar{o}_n R_n + (1-\alpha/\bar{o}_n)\alpha/\bar{o}_n $
$ R_{n-1} + (1-\alpha/\bar{o}_{n-1})^{2}R_{n-2} + ... + $
$ (1-\alpha/\bar{o}_{2})^{n-1}R_1 + (1-\alpha/\bar{o}_1)^{n}Q_1 $

$ Q_{n+1}= \sum_{i=1}^n(1-\alpha/\bar{o}_{n-i})^iR_{n-i} + (1-\alpha)^n Q_1 $

We can see that our bracket with Reward gets bigger since at every timestep we subtract less and less.

### 2.8
*UCB Spikes*   In Figure 2.4 the UCB algorithm shows a distinct spike
in performance on the 11th step. Why is this? Note that for your answer to be fully
satisfactory it must explain both why the reward increases on the 11th step and why it
decreases on the subsequent steps. Hint: If c = 1, then the spike is less prominent. 

It is because our algorithm happened to check all of values since in the beginning all the values
were uncertain, so at this step it came back to the value and since c is 2 the uncertainity is big for others so it goes back to them for a few steps

### 2.9
Show that in the case of two actions, the soft-max distribution is the same
as that given by the logistic, or sigmoid, function often used in statistics and artificial
neural networks.

Suppose we have constant rewards for two different actions a and b: $r_{a}$ and $r_{b}$:  
Lets see value of a for softmax:
$ Pr\{A_{t} = a\}~=~\frac{e^{H_{t}(a)}}{e^{H_{t}(a)}+e^{H_t(b)}} $  
$ Pr\{A_{t} = b\}~=~\frac{e^{H_{t}(b)}}{e^{H_{t}(a)}+e^{H_t(b)}} $  
And for sigmoid:  
$ Pr\{A_{t} = a\}~=~\frac{1}{1+e^{-a}} $  
$ Pr\{A_{t} = b\}~=~\frac{1}{1+e^{-b}} $  
$ \frac{1}{1+e^{-a}}~=~\frac{e^{H_{t}(a)}}{e^{H_{t}(a)}+e^{H_t(b)}} $  

### 2.10
Suppose you face a 2-armed bandit task whose true action values change
randomly from time step to time step. Specifically, suppose that, for any time step,
the true values of actions 1 and 2 are respectively 10 and 20 with probability 0.5 (case
A), and 90 and 80 with probability 0.5 (case B). If you are not able to tell which case
you face at any step, what is the best expected reward you can achieve and how should
you behave to achieve it? Now suppose that on each step you are told whether you are
facing case A or case B (although you still don’t know the true action values). This is an
associative search task. What is the best expected reward you can achieve in this task,
and how should you behave to achieve it?

1) Our model will probably pick these actions randomly since both of them will have the same mean if these cases occur on flip of a coin distribution
2) If we'll know which case are we facing at every timestep then our model can learn the best possible action at every time step.
3) We will have to learn the best values for each case and our best possible average reward can be $\frac{90+20}{2}=55$

### 2.11
Make a figure analogous to Figure 2.6 for the nonstationary
case outlined in Exercise 2.5. Include the constant-step-size $\epsilon$-greedy algorithm with
$\alpha= 0.1$. Use runs of 200,000 steps and, as a performance measure for each algorithm and
parameter setting, use the average reward over the last 100,000 steps.

In [ ]:
#2.5
import numpy as np
from numpy import zeros, argmax, random, empty, linspace
from numpy.random import normal, randint, random as rand
steps, episodes, alpha, n_levers = 20000, 2000, 0.1, 10
epsilon = linspace(0,0.5,10)
Qs = np.array([np.zeros(n_levers) for _ in range(episodes)])
ns = np.array([np.zeros(n_levers) for _ in range(episodes)])
qs = np.array([[normal(randint(-2,3),1) for _ in range(n_levers)] for _ in range(episodes)])
avgrwd = np.array([zeros(steps, dtype=float) for _ in range(episodes)])
rewards = np.empty(len(epsilon))
bstrwd = np.array([zeros(steps, dtype=float) for _ in range(episodes)])
best_rewards = np.empty(len(epsilon))

for ieps, eps in enumerate(epsilon):
    for e in range(episodes):
        for i in range(steps):
            act = argmax(Qs[e]) if eps < rand() else randint(n_levers)
            ns[e][act] += 1
            Qs[e][act] += (qs[e][act] - Qs[e][act]) / ns[e][act]
            avgrwd[e][i] = (avgrwd[e][i-1] * i + qs[e][act]) / (i+1)
        #    print(f"act={act}, Qs[act]={Qs[act]:+.5f}, argmax(Qs)={argmax(Qs)}\
        #, argmax(qs)={argmax(qs)}, max(qs)={max(qs):+.5f}, avgrwd[i]={avgrwd[i]:+.5f}, best action chosen: {act == argmax(qs)}")
            bstrwd[e][i] = max(qs[e])
            for j in range(len(qs[e])): qs[e][j] += normal(0, 0.01)
    ns.fill(0); Qs.fill(0); 
    rewards[ieps] = np.mean(avgrwd[:,int(steps/2):], axis=(0,1))
    best_rewards[ieps] = np.mean(bstrwd[:,int(steps/2):], axis=(0,1))

![Rozwiazanie](2_11.jpg)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(epsilon, rewards, c='r', label='Average Reward')
ax.legend()
ax.set_xlabel("eps")
ax.set_ylabel("Reward")
plt.savefig("2_11.jpg")
plt.show()